In [ ]:
blender_version = '5.0.1' #@param ['5.0.1', '5.1.0'] {allow-input: true}
blend_file_path = 'path/to/file.blend' #@param {type: 'string'}
#@markdown ---
url_blend = '' #@param {type: 'string'}
#@markdown ---
animation = False #@param {type: 'boolean'}
start_frame =  1#@param {type: 'integer'}
end_frame =  250#@param {type: 'integer'}
#@markdown ---
output_name = 'blender-##' #@param {type: 'string'}
zip_files = True #@param {type: 'boolean'}
drive_output_path = 'blender/output' #@param {type: 'string'}
#@markdown ---
gpu_enabled = True #@param {type:"boolean"}
optix_enabled = False #@param {type:"boolean"}
cpu_enabled = False #@param {type:"boolean"}

In [ ]:
%cd /content

gpu = !nvidia-smi --query-gpu=gpu_name --format=csv,noheader
print("Current GPU: " + gpu[0])

if gpu[0] == "Tesla K80" and optix_enabled:
  print("OptiX disabled because of unsupported GPU")
  optix_enabled = False

In [ ]:
import os

os.environ["LD_PRELOAD"] = ""

!apt remove libtcmalloc-minimal4
!apt install libtcmalloc-minimal4

os.environ["LD_PRELOAD"] = "/usr/lib/x86_64-linux-gnu/libtcmalloc_minimal.so.4.5.9"

In [ ]:
import shutil
from google.colab import files, drive

# Mount Google Drive
drive.mount('/drive')

# Download file from URL
uploaded_filename = ""
!wget -nc $url_blend
uploaded_filename = os.path.basename(url_blend)

In [ ]:
!rm -r render
!mkdir render
!mkdir -p trash

if uploaded_filename.lower().endswith('.blend'):
    shutil.copy(uploaded_filename, 'render/')
    blend_file_path = uploaded_filename
else:
    raise SystemExit("Invalid file extension, only .blend files can be uploaded.")

In [ ]:
import requests

major_minor = ".".join(blender_version.split('.')[:2])
blender_url = f"https://ftp.nluug.nl/pub/graphics/blender/release/Blender{major_minor}/blender-{blender_version}-linux-x64.tar.xz"

try:
    response = requests.head(blender_url, allow_redirects=True, timeout=10)
    if response.status_code != 200:
        print(f"Download failed for version '{blender_version}'.")
        print("Error downloading: You may need to define the download archive manually above.")
    else:
        base_url = os.path.basename(blender_url)
        print(f"Download URL: {blender_url}")
        print(f"Base filename: {base_url}")
except Exception as e:
    print(f"Error checking URL: {e}")
    print("Error downloading: You may need to define the download archive manually above.")

!mkdir $blender_version
!wget -nc $blender_url
!tar -xkf $base_url -C ./$blender_version --strip-components=1

In [ ]:
# Enable GPU rendering (or add custom properties here)
data = "import re\n"+\
    "import bpy\n"+\
    "scene = bpy.context.scene\n"+\
    "scene.cycles.device = 'GPU'\n"+\
    "prefs = bpy.context.preferences\n"+\
    "prefs.addons['cycles'].preferences.get_devices()\n"+\
    "cprefs = prefs.addons['cycles'].preferences\n"+\
    "print(cprefs)\n"+\
    "for compute_device_type in ('CUDA', 'OPENCL', 'NONE'):\n"+\
    "    try:\n"+\
    "        cprefs.compute_device_type = compute_device_type\n"+\
    "        print('Device found:',compute_device_type)\n"+\
    "        break\n"+\
    "    except TypeError:\n"+\
    "        pass\n"+\
    "for device in cprefs.devices:\n"+\
    "    if not re.match('intel', device.name, re.I):\n"+\
    "        print('Activating',device)\n"+\
    "        device.use = "+str(gpu_enabled)+"\n"+\
    "    else:\n"+\
    "        device.use = "+str(cpu_enabled)+"\n"
with open('setgpu.py', 'w') as f:
    f.write(data)

renderer = "CUDA"
if optix_enabled:
    print("Note: You're currently using OptiX renderer. If an error occurred, the current GPU (e.g. Tesla K80) is not supported and you need to switch back to CUDA.")
    renderer = "OPTIX"

In [ ]:
%cd /content

!rm -r output
!mkdir output

if not drive_output_path.endswith('/'):
    drive_output_path += '/'

output_path = '/drive/MyDrive/' + drive_output_path + output_name

%cd /content/$blender_version

if animation:
    if start_frame == end_frame:
        !./$blender_version/blender -b '/content/render/{blend_file_path}' -P "/content/setgpu.py" -E CYCLES -o '{output_path}' -noaudio -a -- --cycles-device "{renderer}"
    else:
        !./$blender_version/blender -b '/content/render/{blend_file_path}' -P "/content/setgpu.py" -E CYCLES -o '{output_path}' -noaudio -s $start_frame -e $end_frame -a -- --cycles-device "{renderer}"
else:
    !./$blender_version/blender -b '/content/render/{blend_file_path}' -P "/content/setgpu.py" -E CYCLES -o '{output_path}' -noaudio -f $start_frame -- --cycles-device "{renderer}"

In [ ]:
%cd /content

path, dirs, files_folder = next(os.walk("output"))
output_folder_name = output_name.replace('#', '') + 'render'

if len(files_folder) == 1:
    render_img = 'output/' + files_folder[0]
    shutil.copy('/content/' + render_img, '/drive/MyDrive/' + drive_output_path)
    print(f"Imagen renderizada guardada en Google Drive: {drive_output_path}{files_folder[0]}")
elif len(files_folder) > 1:
    if zip_files:
        shutil.make_archive(output_folder_name, 'zip', 'output')
        shutil.copy('/content/' + output_folder_name + ".zip", '/drive/MyDrive/' + drive_output_path)
        print(f"Archivo ZIP guardado en Google Drive: {drive_output_path}{output_folder_name}.zip")
    else:
        for f in files_folder:
            shutil.copy("/content/output/" + f, '/drive/MyDrive/' + drive_output_path + f)
        print(f"Archivos renderizados guardados en Google Drive: {drive_output_path}")
else:
    raise SystemExit("No frames are rendered.")

# Clear trash folder
trash_path = '/content/trash'
if os.path.exists(trash_path):
    shutil.rmtree(trash_path)
    os.makedirs(trash_path)
    print("Trash folder cleared successfully.")

## Disclaimer
Google Colab is targeted to researchers and students to run AI/ML tasks, data analysis and education, not rendering 3D scenes. Because the computing power provided are free, the usage limits, idle timeouts and speed of the rendering may varies time by time. [Colab Pro and Colab Pro+](https://colab.research.google.com/signup) are available for those who wanted to have more powerful GPU and longer runtimes for rendering. See the [FAQ](https://research.google.com/colaboratory/faq.html) for more info. In some cases, it might be faster to use an online Blender renderfarm.

## License
```
MIT License

Copyright (c) 2020-2022 ynshung

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
```